<a href="https://colab.research.google.com/github/Lidivinekeng/Prediction_of_product_sales/blob/main/lidivine_kengne_sales_prediction_project.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

#prediction of product sales



*   Author : Lidivine Kengne



## project overview

##load and inspect data

##clean data

##exploratory data analysis

##feature inspection